In [1]:
import pandas as pd
import glob
import os

In [2]:
folder_path = 'data/*.csv'
dfs = []
for file_path in glob.glob(folder_path):
    df = pd.read_csv(file_path)
    file_name = os.path.basename(file_path)  # Get the file name from the file path
    df_subset = df[['Total State Funds*', 'State Agencies']]  # Select the desired column(s)
    df_subset['Year'] = file_name[:4]  # Add a column with the file name
    dfs.append(df_subset)

combined_df = pd.concat(dfs, ignore_index=True).sort_values(by='Year').reset_index(drop=True)

In [3]:
combined_df

,Total State Funds*,State Agencies,Year
0,"$3,186,058",Environmental Protection,2017
1,"$183,256,293",TOTALS,2017
2,"$12,988,956",Transportation,2017
3,"$5,197,052",Natural Resources,2017
4,"$6,744,679","Legislative, Judicial, and Executive",2017
...,...,...,...
86,"$1,916,824",Labor and Workforce Development,2023
87,"$10,329,964","Legislative, Judicial, and Executive",2023
88,"$11,256,487",Natural Resources,2023
89,"$18,900,398",Transportation,2023


In [4]:
df = combined_df
# Remove dollar signs and convert to integer
df['Total State Funds*'] = df['Total State Funds*'].replace({'\$': '', ',': ''}, regex=True).astype(int)

# Exclude the "TOTALS" row
df = df[df['State Agencies'] != 'TOTALS']

df.head()

df['State Agencies'].unique()

df.loc[df['State Agencies'] == 'Business, Consumer Services, and Housing', 'State Agencies'] = 'Business, Cons. Serv. & Housing'

In [5]:
import altair as alt

# Line Chart
line_chart = alt.Chart(df).mark_line(point=True).encode(
    x='Year:O',
    y=alt.Y('Total State Funds*', title='Total State Funds in thousands of dollars'),
    color='State Agencies',
    tooltip=['Year', 'State Agencies', 'Total State Funds*']
).properties(
    width=700,
    height=400,
    title='Yearly State Funding by Agency (Line Chart)'
)
line_chart.interactive()

alt.Chart(...)

In [6]:
# Stacked Bar Chart
stacked_bar_chart = alt.Chart(df).mark_bar().encode(
    x='Year:O',
    y=alt.Y('Total State Funds*', title='State Funds in thousands of dollars'),
    color='State Agencies:N',
    tooltip=['Year', 'State Agencies', 'Total State Funds*']
).properties(
    width=700,
    height=400,
    title='Yearly State Funding by Agency (Stacked Bar Chart)'
)

stacked_bar_chart

alt.Chart(...)

In [7]:
!pip install openpyxl

In [8]:
import openpyxl
data = pd.read_excel("/home/jovyan/work/California Department Spending.xlsx")

# Set the first row as the header
data.columns = data.iloc[0]

# Drop the first row
data = data.iloc[1:]

# Reset the index
data.reset_index(drop=True, inplace=True)

# Display the cleaned data
data.head()

,index,State Agencies,2017 Departments,2022 Departments,2017 Spending,2022 Spending,Percent Change
0,0,"Business, Consumer Services, Housing","Business, Consumer Svcs, Housing, Scty","Business, Consumer Svcs, Housing, Scty",763,1307287,1712.351245
1,1,"Business, Consumer Services, Housing",Cannabis Control Appeals Panel,Cannabis Control Appeals Panel,1045,3199,2.061244
2,2,"Business, Consumer Services, Housing",Department of Consumer Affairs,Department of Consumer Affairs,653307,696177,0.06562
3,3,"Business, Consumer Services, Housing",NaN,Department of Cannabis Control,NaN,200880,NaN
4,4,"Business, Consumer Services, Housing",NaN,"Department of Fair Employment, Housing",NaN,49240,NaN


In [9]:

# Convert the '2017 Spending' and '2022 Spending' columns to numeric format
data['2017 Spending'] = pd.to_numeric(data['2017 Spending'], errors='coerce')
data['2022 Spending'] = pd.to_numeric(data['2022 Spending'], errors='coerce')

# Multiply the spending columns by 1000 to add three zeroes
data['2017 Spending'] *= 1000
data['2022 Spending'] *= 1000

# Combine '2017 Departments' and '2022 Departments' into a single 'Department' column
data['Department'] = data['2017 Departments'].fillna(data['2022 Departments'])

# Create a new dataframe where each row is a department-year and the columns are 'Department', 'Year', 'Spending'
data_long = data.melt(
    id_vars=['Department'],
    value_vars=['2017 Spending', '2022 Spending'],
    var_name='Year',
    value_name='Spending'
)

# Map '2017 Spending' and '2022 Spending' to '2017' and '2022'
data_long['Year'] = data_long['Year'].map({'2017 Spending': '2017', '2022 Spending': '2022'})

# Sort the departments by 2022 spending and take the top 20
top_departments = (
    data_long[data_long['Year'] == '2022']
    .groupby('Department')
    .sum()
    .sort_values('Spending', ascending=False)
    .head(30)
    .index
    .tolist()
)

# Filter data_long to include only the top 20 departments from 2022
data_long = data_long[data_long['Department'].isin(top_departments)]

top_departments

['Department of Education',
 'Department of Health Care Services',
 'Department of Social Services',
 'Corrections and Rehabilitation',
 'Board of Governors of Community Colleges',
 'Department of Transportation',
 'Department of Developmental Services',
 'Various Departments',
 'State-Local Realignment, 1991',
 'State-Local Realignment, 2011',
 'University of California',
 'Judicial Branch',
 'California State University',
 'Department of Public Health',
 'Other General Government Departments',
 'Teachers Retirement System Contributions',
 'Shared Revenues',
 'Department of Forestry, Fire Protection',
 'Student Aid Commission',
 'Dept of Housing, Community Development',
 'Dept of the California Highway Patrol',
 'Department of State Hospitals',
 "Public Employees' Retirement System",
 'General Obligation Bonds-K-12',
 'Health, Dental Benefits for Annuitants',
 'Resources Recycling and Recovery',
 'Air Resources Board',
 'Department of Water Resources',
 'Local Community Corrections',


In [10]:



# Create the chart
chart_2017 = (
    alt.Chart(data_long[data_long['Year'] == '2017'])
    .mark_bar()
    .encode(
        x=alt.X('Spending:Q', title='Spending'),
        y=alt.Y(
            'Department:N',
            title='Department',
            sort=top_departments
        ),
        color=alt.Color('Year:N'),
        opacity=alt.value(0.3),
        tooltip=[
            alt.Tooltip('Department', title='Department'),
            alt.Tooltip('Year', title='Year'),
            alt.Tooltip('Spending', title='Spending', format='$,.2f')
        ]
    )
)

chart_2022 = (
    alt.Chart(data_long[data_long['Year'] == '2022'])
    .mark_bar()
    .encode(
        x=alt.X('Spending:Q', title='Spending'),
        y=alt.Y(
            'Department:N',
            title='Department',
            sort=top_departments
        ),
        color='Year:N',
        opacity=alt.value(0.5),
        tooltip=[
            alt.Tooltip('Department', title='Department'),
            alt.Tooltip('Year', title='Year'),
            alt.Tooltip('Spending', title='Spending', format='$,.2f')
        ]
    )
)

chart = alt.layer(chart_2022, chart_2017).interactive()
chart

alt.LayerChart(...)

In [11]:
import pandas as pd

all_sheets = pd.read_excel("Historical_Expenditures.xlsx", sheet_name=None)
print(all_sheets.keys())

dict_keys(['Pivot Table', 'Pivot Table Data'])


In [12]:
df = all_sheets['Pivot Table Data']

#df

In [13]:
df = df.drop(columns=[col for col in df.columns if 'Unnamed:' in col])
# Drop rows with missing 'Amount' data
df = df.dropna(subset=['Amount'])

# Convert 'Fiscal Year' to datetime
df['Fiscal Year'] = df['Fiscal Year'].apply(lambda x: pd.to_datetime(x.split('-')[0]))

# Exclude Federal Funds
df_non_federal = df[df['Fund'] != 'Federal Funds']




In [14]:
# Define the old strings to replace and the new string
old_strings = ['Department of Health Services', 'State Department of Health Care Services']
new_string = 'Department of Health Care Services'

old_strings2 = ['Department of Corrections', 'Department of Corrections & Rehabilitation']
new_string2 = 'Corrections and Rehabilitation'


# Replace the old strings with the new string
df_non_federal = df_non_federal.replace(old_strings, new_string)
df_non_federal = df_non_federal.replace(old_strings2, new_string2)

df_non_federal.head()


,Dept. Code,Department,Function,Fiscal Year,Fund,DOF Agency,Debt Service?,Amount
0,1110,Department of Consumer Affairs - Regulatory Bo...,State Operations,1984-01-01,General Fund,"Business, Consumer Services, and Housing",Not GO Debt Service,1817.0
1,1110,Department of Consumer Affairs - Regulatory Bo...,State Operations,1984-01-01,Special Funds,"Business, Consumer Services, and Housing",Not GO Debt Service,92824.0
4,1110,Department of Consumer Affairs - Regulatory Bo...,State Operations,1985-01-01,General Fund,"Business, Consumer Services, and Housing",Not GO Debt Service,1966.0
5,1110,Department of Consumer Affairs - Regulatory Bo...,State Operations,1985-01-01,Special Funds,"Business, Consumer Services, and Housing",Not GO Debt Service,103422.0
8,1110,Department of Consumer Affairs - Regulatory Bo...,State Operations,1986-01-01,General Fund,"Business, Consumer Services, and Housing",Not GO Debt Service,1960.0


In [15]:

# Group by 'Fiscal Year' and 'Department' and calculate the total 'Amount'
agency_grouped = df_non_federal.groupby(['Fiscal Year', 'DOF Agency'])['Amount'].sum().reset_index()

agencydata = alt.Data(values=agency_grouped.to_dict(orient='records'))

# Create a stacked area chart
chart = alt.Chart(agencydata).mark_area().encode(
    x='Fiscal Year:T',
    y=alt.Y('Amount:Q', stack='zero'),  # Use 'stack="zero"' for non-normalized
    color=alt.Color('DOF Agency:N'),
    tooltip=['Fiscal Year:T', 'DOF Agency:N', 'Amount:Q']
).properties(
    title='Total Expenditures Over Time Excluding Federal Funds'
)

chart.interactive()

alt.Chart(...)

In [16]:

# Group by 'Fiscal Year' and 'Department' and calculate the total 'Amount'
df_grouped = df_non_federal.groupby(['Fiscal Year', 'Department'])['Amount'].sum().reset_index()

data = alt.Data(values=df_grouped.to_dict(orient='records'))

# Create a stacked area chart
chart = alt.Chart(data).mark_area().encode(
    x='Fiscal Year:T',
    y=alt.Y('Amount:Q', stack='zero'),  # Use 'stack="zero"' for non-normalized
    color=alt.Color('Department:N', legend = None),
    tooltip=['Fiscal Year:T', 'Department:N', 'Amount:Q']
).properties(
    title='Total Expenditures Over Time Excluding Federal Funds'
)

chart.interactive()

alt.Chart(...)

In [17]:
import pandas as pd

all_sheets = pd.read_excel("Historical_Revenues (1).xlsx", sheet_name=None)
revenues = all_sheets['Pivot Table Raw Data'] 

revenues

,Unnamed: 0,Category,Code,Source,Type of Fund,Fiscal Year,Amount,Unnamed: 7
0,1,Major Revenue,1,Alcoholic Beverages Taxes & Fees,General Fund,1950-51,1.989005e+04,NaN
1,2,Major Revenue,2,Corporation Tax,General Fund,1950-51,9.824521e+04,NaN
2,3,Major Revenue,3,Cigarette Tax,General Fund,1950-51,0.000000e+00,NaN
3,4,Major Revenue,4,Horse Racing (Pari-mutual License Fees),General Fund,1950-51,3.900476e+03,NaN
4,5,Major Revenue,5,"Estate, Inheritance & Gift Tax",General Fund,1950-51,2.367102e+04,NaN
...,...,...,...,...,...,...,...,...
3457,3418,Major Revenue,14,Retail Sales and Use Tax,General Fund,2022-23,3.399200e+07,NaN
3458,3419,Major Revenue,2,Corporation Tax,General Fund,2022-23,3.846400e+07,NaN
3459,3420,Major Revenue,6,Insurance Gross Premium Tax,General Fund,2022-23,3.667000e+06,NaN
3460,3421,Minor Revenue,27,All Other Minor Revenue,General Fund,2022-23,9.046000e+06,NaN


In [18]:
revenues = revenues.drop(columns=[col for col in df.columns if 'Unnamed:' in col])
# Drop rows with missing 'Amount' data
revenues = revenues.dropna(subset=['Amount'])

# Convert 'Fiscal Year' to datetime
revenues['Fiscal Year'] = revenues['Fiscal Year'].apply(lambda x: pd.to_datetime(x.split('-')[0]))


In [20]:
dfallrevenues = revenues[(revenues['Fiscal Year'].dt.year >= 1985) & (revenues['Fiscal Year'].dt.year <= 2017)]

dfallrevenues

,Unnamed: 0,Category,Code,Source,Type of Fund,Fiscal Year,Amount,Unnamed: 7
1750,1751,Major Revenue,1,Alcoholic Beverages Taxes & Fees,General Fund,1985-01-01,132261.0,NaN
1751,1752,Major Revenue,2,Corporation Tax,General Fund,1985-01-01,3843024.0,NaN
1752,1753,Major Revenue,3,Cigarette Tax,General Fund,1985-01-01,181252.0,NaN
1753,1754,Major Revenue,4,Horse Racing (Pari-mutual License Fees),General Fund,1985-01-01,111933.0,NaN
1754,1755,Major Revenue,5,"Estate, Inheritance & Gift Tax",General Fund,1985-01-01,252811.0,NaN
...,...,...,...,...,...,...,...,...
3427,3388,Minor Revenue,28,All Other Minor Revenue,Special Funds,2017-01-01,21463035.0,NaN
3428,3389,Transfers & Loans,29,Transfers & Loans,General Fund,2017-01-01,-2295260.0,NaN
3429,3390,Transfers & Loans,29,Transfers & Loans,Special Funds,2017-01-01,2275974.0,NaN
3430,3391,SCO Adjustments,30,Adjustments to Reconcile to Controller,General Fund,2017-01-01,0.0,NaN


In [21]:
import altair as alt

# Group by 'Fiscal Year' and 'Department' and calculate the total 'Amount'
rev_grouped = dfallrevenues.groupby(['Fiscal Year', 'Source'])['Amount'].sum().reset_index()

data2 = alt.Data(values=rev_grouped.to_dict(orient='records'))

# Create a stacked area chart
chart = alt.Chart(data2).mark_area().encode(
    x='Fiscal Year:T',
    y=alt.Y('Amount:Q', stack='zero'),  # Use 'stack="zero"' for non-normalized
    color=alt.Color('Source:N', legend = None),
    tooltip=['Fiscal Year:T', 'Source:N', 'Amount:Q']
).properties(
    title='Total Revenues Over Time By Source'
)

chart.interactive()

alt.Chart(...)

In [22]:
df_general = revenues[revenues['Type of Fund'] == 'General Fund']

# Group by 'Fiscal Year' and 'Department' and calculate the total 'Amount'
gen_grouped = df_general.groupby(['Fiscal Year', 'Source'])['Amount'].sum().reset_index()

data3 = alt.Data(values=gen_grouped.to_dict(orient='records'))

# Create a stacked area chart
chart = alt.Chart(data3).mark_area().encode(
    x='Fiscal Year:T',
    y=alt.Y('Amount:Q', stack='zero'),  # Use 'stack="zero"' for non-normalized
    color=alt.Color('Source:N'),
    tooltip=['Fiscal Year:T', 'Source:N', 'Amount:Q']
).properties(
    title='Total Revenues Over Time By Source'
)

chart.interactive()

alt.Chart(...)

In [23]:

df_general85 = df_general[(df_general['Fiscal Year'].dt.year >= 1985)]

# Group by 'Fiscal Year' and 'Department' and calculate the total 'Amount'
gen_grouped = df_general85.groupby(['Fiscal Year', 'Source'])['Amount'].sum().reset_index()

data4 = alt.Data(values=gen_grouped.to_dict(orient='records'))

# Create a stacked area chart
chart = alt.Chart(data4).mark_area().encode(
    x='Fiscal Year:T',
    y=alt.Y('Amount:Q', stack='zero'),  # Use 'stack="zero"' for non-normalized
    color=alt.Color('Source:N', legend = None
                   ),
    tooltip=['Fiscal Year:T', 'Source:N', 'Amount:Q']
).properties(
    title='Total Revenues Over Time By Source'
)

chart.interactive()

alt.Chart(...)

In [24]:
# Calculate total revenues per fiscal year
revenues_per_year = revenues.groupby(['Fiscal Year'])['Amount'].sum().reset_index()
revenues_per_year = revenues_per_year[(revenues_per_year['Fiscal Year'].dt.year <= 2017)]

# Group by 'Fiscal Year' and 'Department' and calculate the total 'Amount'
expenditures_per_year = df_non_federal.groupby(['Fiscal Year'])['Amount'].sum().reset_index()

GDPdata = pd.read_excel("Table (1).xlsx")


#data = alt.Data(values=df_grouped.to_dict(orient='records'))

GDPdata = GDPdata.T
                      
GDPdata.columns = GDPdata.iloc[0]
GDPdata = GDPdata[1:].reset_index()
GDPdata['Fiscal Year'] = GDPdata['index'].apply(lambda x: pd.to_datetime(x.split('-')[0]))
GDPdata = GDPdata[['Fiscal Year','GDP 2022 Dollars','GDP Chained 2012 Dollars']]

result = GDPdata.merge(expenditures_per_year, on='Fiscal Year', how='outer').merge(revenues_per_year, on='Fiscal Year', how='outer')

result = result.rename(columns={'Amount_x': 'Total CA Expenditures', 'Amount_y': 'Total CA Revenues'})

result['GDP 2022 Dollars'] *= 1000
result['GDP Chained 2012 Dollars'] *= 1000

result

,Fiscal Year,GDP 2022 Dollars,GDP Chained 2012 Dollars,Total CA Expenditures,Total CA Revenues
0,1998-01-01,1147520400.0,1468730800.0,75252875.0,74280887.00
1,1999-01-01,1241899700.0,1574305900.0,84858734.0,87535733.00
2,2000-01-01,1356975400.0,1696172400.0,96379144.0,88419043.00
3,2001-01-01,1375761300.0,1692324100.0,99219565.0,89804090.00
4,2002-01-01,1418429600.0,1722522400.0,106778007.0,95794300.00
...,...,...,...,...,...
69,1979-01-01,NaN,NaN,NaN,20918820.24
70,1980-01-01,NaN,NaN,NaN,22103788.00
71,1981-01-01,NaN,NaN,NaN,23600883.00
72,1982-01-01,NaN,NaN,NaN,24291063.00


In [29]:
import altair as alt


# Melt the data into a long format
melted_df = result.melt('Fiscal Year', var_name='a', value_name='b')

# Create a line chart
chart = alt.Chart(melted_df).mark_line().encode(
    x='Fiscal Year:T',
    y='b:Q',
    color='a:N'
)

chart.interactive()

alt.Chart(...)

In [26]:
# Importing necessary libraries
import pandas as pd

# Loading the data
data = pd.read_csv('60000_1997_2022.csv')

# Displaying the first few rows of the data
#data.head()

In [27]:
# Removing rows with NaN values
data = data.dropna()

# Removing the '$' sign and converting the GDP values to floats
for year in range(1997, 2022):
    data[str(year)] = data[str(year)].replace('[\$,]', '', regex=True).astype(float)

# Melting the DataFrame to get 'Year', 'Industry' and 'GDP' columns
data_melted = data.melt(id_vars=['Level1', 'Level2', 'Level3'], var_name='Year', value_name='GDP')

# Renaming the 'Level3' column to 'Industry'
data_melted = data_melted.rename(columns={'Level3': 'Industry'})

# Displaying the first few rows of the preprocessed data
data_melted.head()

,Level1,Level2,Industry,Year,GDP
0,Private industries,"Agriculture, forestry, fishing...",Farms,1997,18853.0
1,Private industries,"Agriculture, forestry, fishing...","Forestry, fishing, and...",1997,4409.9
2,Private industries,"Mining, quarrying, and oil and...",Oil and gas extraction,1997,17890.4
3,Private industries,"Mining, quarrying, and oil and...",Mining (except oil and...,1997,3203.5
4,Private industries,"Mining, quarrying, and oil and...",Support activities for...,1997,753.8


In [28]:
# Importing necessary library
import altair as alt

# Creating the stacked area chart
chart = alt.Chart(data_melted).mark_area().encode(
    x=alt.X('Year:O', title='Year'),
    y=alt.Y('GDP:Q', title='GDP', stack='zero'),
    color=alt.Color('Industry:N', title='Industry', legend=None),
    tooltip=['Industry', 'Year', 'GDP']
).properties(
    title='California GDP by Industry (1997 - 2022)',
    width=600,
    height=400
)

chart.interactive()

alt.Chart(...)

data from https://united-states.reaproject.org/data-tables/gsp-a900n/tools/60000/